<a href="https://colab.research.google.com/github/Ikmalrl/Computer-Security/blob/main/CyberStone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import hashlib
import struct
import math
from typing import List

# ---------- Utility Functions ----------

def pkcs7_pad(data: bytes, block_size: int = 8) -> bytes:
    """Apply PKCS#7 padding to data."""
    pad_len = block_size - (len(data) % block_size)
    return data + bytes([pad_len] * pad_len)

def pkcs7_unpad(data: bytes) -> bytes:
    """Remove PKCS#7 padding."""
    pad_len = data[-1]
    if pad_len < 1 or pad_len > len(data):
        raise ValueError("Invalid padding")
    return data[:-pad_len]


# ---------- Subkey Generation ----------

def generate_round_keys(master_key: bytes, rounds: int = 8) -> List[int]:
    """Generate round subkeys using SHA-256(key + round_number)."""
    round_keys = []
    for i in range(rounds):
        digest = hashlib.sha256(master_key + i.to_bytes(1, 'big')).digest()
        subkey = int.from_bytes(digest[:6], 'big')  # 48-bit subkey
        round_keys.append(subkey)
    return round_keys


# ---------- Round Function ----------

def round_function(right: int, subkey: int) -> int:
    """
    Nonlinear round function:
    1. Mix with subkey
    2. Apply small substitution using nibbles
    3. Bit rotation
    """
    x = (right ^ subkey) & 0xFFFFFFFF
    sbox = [0x6, 0x4, 0xC, 0x5, 0x0, 0x7, 0x2, 0xE,
            0x1, 0xF, 0x3, 0xD, 0x8, 0xA, 0x9, 0xB]
    out = 0
    for i in range(8):  # process 8 nibbles
        nibble = (x >> (i * 4)) & 0xF
        out |= (sbox[nibble] << (i * 4))
    # rotate left 5 bits
    return ((out << 5) | (out >> (32 - 5))) & 0xFFFFFFFF


# ---------- Core Cipher Class ----------

class WingsDings:
    NAME = "WingsDings"

    def __init__(self, key: bytes):
        if len(key) < 16:
            raise ValueError("Key must be at least 16 bytes (128 bits).")
        self.master_key = key[:16]
        self.round_keys = generate_round_keys(self.master_key)

    def encrypt_block(self, block: bytes) -> bytes:
        """Encrypt a single 8-byte block."""
        L, R = struct.unpack(">II", block)
        for i in range(8):
            temp = L ^ round_function(R, self.round_keys[i])
            L, R = R, temp
        return struct.pack(">II", L, R)

    def decrypt_block(self, block: bytes) -> bytes:
        """Decrypt a single 8-byte block."""
        L, R = struct.unpack(">II", block)
        for i in reversed(range(8)):
            temp = R ^ round_function(L, self.round_keys[i])
            R, L = L, temp
        return struct.pack(">II", L, R)

    def encrypt(self, plaintext: bytes) -> bytes:
        """Encrypt arbitrary-length plaintext."""
        padded = pkcs7_pad(plaintext, 8)
        ciphertext = b''
        for i in range(0, len(padded), 8):
            block = padded[i:i+8]
            ciphertext += self.encrypt_block(block)
        return ciphertext

    def decrypt(self, ciphertext: bytes) -> bytes:
        """Decrypt arbitrary-length ciphertext."""
        decrypted = b''
        for i in range(0, len(ciphertext), 8):
            block = ciphertext[i:i+8]
            decrypted += self.decrypt_block(block)
        return pkcs7_unpad(decrypted)


# ---------- Demo Section ----------

if __name__ == "__main__":
    print("\n===== CyberStone Block Cipher =====")

    # User or sample input
    plaintext = input("Enter plaintext: ").encode()
    key = b"wingsdings-secret-key"

    # Initialize cipher
    cipher = WingsDings(key)

    # Encrypt and decrypt
    ciphertext = cipher.encrypt(plaintext)
    decrypted = cipher.decrypt(ciphertext)

    # Display results
    print("\nStatus          : Encryption complete!")
    print("Ciphertext (hex):", ciphertext.hex())
    print("Decrypted text  :", decrypted.decode(errors='ignore'))

    # Verification
    if decrypted == plaintext:
        print("\nSuccess: Decryption matches with the plaintext")
    else:
        print("\nError: Decryption failed.")


===== CyberStone Block Cipher =====
Enter plaintext: computer security

Status          : Encryption complete!
Ciphertext (hex): e632b890ddfb198283f70679847424c4d7d37c84b7bb3892
Decrypted text  : computer security

Success: Decryption matches with the plaintext
